# Do not use this file

In [10]:
from sqlalchemy import create_engine
import pandas as pd
import json
import urllib.parse
import psycopg2
import os 
with open("credentials.json") as file:
    login = json.load(file)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

engine = create_engine("postgresql://{}:{}@{}:{}/postgres".format(username,password,host,port))
path = "charles/2020/CHARLS2020r/"

In [12]:
# this script is using to establish sql database

files = os.listdir(path)
with engine.begin() as connection:    
    for file in files:
        df = pd.read_stata(path + file)
        df.columns = [col.replace('ID', 'UID') if 'ID' in col else col for col in df.columns]
        df.to_sql(name = file[:-4], con = connection, index=False,if_exists='replace')


In [5]:
df = pd.read_stata(path+"COVID_Module.dta")

In [15]:
from sqlalchemy import MetaData, Table

metadata = MetaData()

for file in files:
    table_to_drop = Table(file[:-4], metadata, autoload_with=engine)
    table_to_drop.drop(engine)